# Train


<h2>Mount Drive</h2>

In [0]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h2>Navigate to Target dataset</h2>

In [0]:
import os
print(os.getcwd())
os.chdir('drive/My Drive/covid_sets')
print(os.getcwd())

/content
/content/drive/My Drive/covid_sets


<h2>Load Target dataset</h2>

In [0]:
import pandas as pd
import numpy as np

#init dict to store target datasets
datasets = {'X_train': None,'y_train':None,'X_val':None,'y_val':None,'X_test':None,'y_test':None}

#find corresponding dataset and add to dict
for file in os.listdir():
  filename = file[:-4]
  print(filename)
  if filename in datasets:
    datasets[filename] = np.load(os.path.join(os.getcwd(),file))

target_metadata
X_train
y_train
X_val
y_val
X_test
y_test


<h2>Load Pretrained Models</h2>

In [0]:
#### pull pretrained models from tf.keras
from keras.applications import MobileNetV2, InceptionV3, ResNet50V2, VGG16, VGG19, Xception, ResNet101V2, DenseNet121, ResNet152V2, DenseNet169, DenseNet201 ##
from keras.models import load_model
from keras.applications import ResNet50V2
from keras.metrics import AUC

#go to tuned models directory
os.chdir('../mid_slices/tuned_models')


#iterate through tuned models and add all unfinalized models to model list
model_list = []
model_name = None
for model in os.listdir():
  if "target" not in model :
    
    model_name = model[:-5]
    print(model_name)
    tuned_model = load_model(model)
    model_list.append(tuned_model)

#display loaded models
[model.summary() for model in model_list]


###uncomment for imagenet baseline model###
# img_shape = (224,224,3)


#model initialization
# densenet121 = DenseNet121(weights='imagenet', include_top=False, input_shape=img_shape)
# densenet169 = DenseNet169(weights='imagenet', include_top=False, input_shape=img_shape)
# resnet50 = ResNet50V2(weights='imagenet', include_top=False, input_shape=img_shape)
# resnet101 = ResNet101V2(weights='imagenet', include_top=False, input_shape=img_shape)



#append models to mdoel list
# model_list = [densenet121,densenet169,resnet50,resnet101]



Using TensorFlow backend.


densenet121_soft_labels
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1025000   
_________________________________________________________________
dense_2 (Dense)              (None, 16)                16016     
Total params: 8,078,520
Trainable params: 5,626,168
Non-trainable params: 2,452,352
_________________________________________________________________


[None]

<h2>Randomly initialize FC layers and freeze shallow half of Conv layers</h2>

In [0]:
##replace last layer and recompile to do binary cross entropy
from keras import models, layers
from keras.layers import Dropout,AveragePooling2D, GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D
from time import sleep

target_models = []
for model in model_list:
  
    #get model name
    name = model.layers[0].name
    print(name)

    #create new sequential model and add fine-tuned convolutional base to new model
    tuned_model = models.Sequential()
    [tuned_model.add(layer) for layer in model.layers[:-2]]
    

    #random initialization fixed for reproducibility
    np.random.seed(1000)
    


    #add randomly initialized output layers with dropout addition
    if "resnet" in name:
    #---------------RESNET LAYERS---------------
      tuned_model.add(layers.Dense(1000, activation='relu'))
      tuned_model.add(Dropout(0.5))
      tuned_model.add(layers.Dense(1, activation='sigmoid'))

    elif "densenet" in name:
    #---------------DENSENET LAYERS---------------
      tuned_model.add(layers.Dense(1000, activation='relu'))
      tuned_model.add(Dropout(0.5))
      tuned_model.add(layers.Dense(1, activation='sigmoid'))

    #print model summary
    print(tuned_model.summary())


    #freeze shallower half of all convolutional layers
    target_model = tuned_model
    #conv layers
    conv_layers = []
    for layer in target_model.layers[0].layers:
        if isinstance(layer,Conv2D):
          layer.trainable = True
          conv_layers.append(layer)

    n_freeze = len(conv_layers)//2
    for layer in conv_layers[:n_freeze]:
        layer.trainable = False 

    for layer in target_model.layers[0].layers:
        # layer.trainable = True
        print(layer,layer.trainable,layer.output_shape)

    #append model to target models
    target_models.append(target_model)
      



    




densenet121
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
Total params: 7,037,504
Trainable params: 4,585,152
Non-trainable params: 2,452,352
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1025000 

<h2>Learn Target task</h2>

In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD, Adadelta, Adam, Adagrad, RMSprop #, PolynomialDecay
from keras.losses import BinaryCrossentropy
from keras.metrics import AUC,Recall

import time

for target_model in target_models:

  #start counter
  start = time.perf_counter()

  #optimizer
  if ("resnet" in target_model.layers[0].name or "densenet"in target_model.layers[0].name):
    opt = SGD(momentum=0.9) 
  else:
    opt = Adadelta()
 
  #create modle save path
  name = target_model.layers[0].name + "_soft_labels_target.hdf5"
  print(target_model.layers[0].name)
  save_path = os.path.join(os.getcwd(),name)

  #compile model
  target_model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])


  #train model and save model with best validation accuracy
  checkpointer = [ModelCheckpoint(filepath=save_path, verbose=1, save_best_only=True, monitor='val_accuracy',mode='max')]
  train_history = target_model.fit(datasets['X_train'], datasets['y_train'],verbose=1,epochs=60,batch_size=32,validation_data=(datasets['X_val'],datasets['y_val']),callbacks=checkpointer,shuffle=True)

  #end counter
  finish = time.perf_counter()
  print(f'this process took {finish-start} seconds')



densenet121
compiled
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              1025000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1001      
Total params: 8,063,505
Trainable params: 5,611,153
Non-trainable params: 2,452,352
_________________________________________________________________
None
Train on 447 samples, validate on 112 samples
Epoch 1/60
447/447 [==========